# Segmenting and Clustering Boroughs in Toronto

## Installing and importing the modules

In [34]:
!pip install folium

In [35]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Prepating the data

### Acquiring the initial dataset

In [36]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url).content
soup = BeautifulSoup(result, 'html.parser')
df = pd.read_html(str(soup.table))[0]

### Formatting the dataframe

In [37]:
# Ignoring the cells with a borough that is Not assigned
df = df[df.Borough != 'Not assigned']

# For the Not assigned neighborhoods, the neighborhood is the same as the borough
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']

# Multiple neighborhoods for the same Postal Code
columns = df.columns.tolist()
columns.remove("Neighborhood")
df = df.groupby(columns)['Neighborhood'].apply(list)
df = df.reset_index()

df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"[Malvern, Rouge]"
1,M1C,Scarborough,"[Rouge Hill, Port Union, Highland Creek]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


## Getting the dataframe's shape

In [38]:
df.shape

(103, 3)

## Getting the coordinates

In [39]:
coordinates = pd.read_csv('http://cocl.us/Geospatial_data')
df = pd.concat([df.set_index('Postal Code'),coordinates.set_index('Postal Code')], axis=1, join='inner').reset_index()
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Malvern, Rouge]",43.806686,-79.194353
1,M1C,Scarborough,"[Rouge Hill, Port Union, Highland Creek]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


## Creating the map

In [40]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

## Nearby venues function (from the lab, using Borough instead of Neighborhood)

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Getting the venues

In [57]:
# The code was removed by Watson Studio for sharing.

In [43]:
LIMIT = 100
radius = 500

toronto_venues = getNearbyVenues(names=df['Borough'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

toronto_venues.head()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

## Creating dummies and grouping

In [44]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Borough'] = toronto_venues['Borough'] 

toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017094,...,0.000000,0.008547,0.000000,0.000000,0.008547,0.000000,0.000000,0.000000,0.000000,0.008547
1,Downtown Toronto,0.000000,0.000826,0.000826,0.000826,0.000826,0.001652,0.001652,0.001652,0.014864,...,0.002477,0.010735,0.001652,0.000000,0.003303,0.000000,0.005780,0.001652,0.000826,0.005780
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008130,0.000000,0.000000,0.024390
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.013158,0.000000,0.013158,0.000000,0.000000,0.000000,0.013158
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013699,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004219,0.000000,0.004219,0.000000,0.000000,0.000000,0.000000,0.000000,0.008439,...,0.000000,0.000000,0.004219,0.004219,0.008439,0.000000,0.000000,0.000000,0.012658,0.000000
7,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010989,...,0.000000,0.000000,0.000000,0.000000,0.010989,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.019481,0.000000,0.000000,0.006494,0.000000,0.006494,0.000000,0.000000,0.006494
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000


## Getting top 5 venues for each borough

In [45]:
num_top_venues = 5

for hood in toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Park  0.05
3            Café  0.05
4     Pizza Place  0.03


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.06
2           Restaurant  0.04
3                Hotel  0.03
4  Japanese Restaurant  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.06
2             Brewery  0.04
3  Italian Restaurant  0.04
4                Café  0.04


----East York----
                 venue  freq
0                 Park  0.05
1          Coffee Shop  0.05
2                 Bank  0.05
3  Sporting Goods Shop  0.04
4         Burger Joint  0.04


----Etobicoke----
                  venue  freq
0           Pizza Place  0.11
1        Sandwich Place  0.07
2  Fast Food Restaurant  0.04
3          Liquor Store  0.04
4                  Park  0.04


----Mississauga----
                      ve

## The function to sort the venues in descending order

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Getting the top 10 venues for each borough

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Restaurant,Dessert Shop,Sushi Restaurant,Pizza Place,Pub,Gym
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Brewery,Italian Restaurant,Café,Restaurant,Ice Cream Shop,Yoga Studio,Park,Pub
3,East York,Park,Coffee Shop,Bank,Sporting Goods Shop,Pharmacy,Pizza Place,Burger Joint,Supermarket,Restaurant,Liquor Store
4,Etobicoke,Pizza Place,Sandwich Place,Fast Food Restaurant,Gym,Pharmacy,Grocery Store,Coffee Shop,Liquor Store,Park,Beer Store
5,Mississauga,Hotel,Coffee Shop,Gym,American Restaurant,Middle Eastern Restaurant,Intersection,Mediterranean Restaurant,Sandwich Place,Fried Chicken Joint,Burrito Place
6,North York,Coffee Shop,Clothing Store,Restaurant,Bank,Sandwich Place,Japanese Restaurant,Park,Grocery Store,Pizza Place,Sushi Restaurant
7,Scarborough,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
8,West Toronto,Café,Bar,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Pizza Place,Breakfast Spot,Bookstore,Grocery Store
9,York,Fast Food Restaurant,Park,Fried Chicken Joint,Pizza Place,Field,Sandwich Place,Bus Line,Restaurant,Caribbean Restaurant,Brewery


## Clustering

In [48]:
kclusters = 5

toronto_grouped_clustering =  toronto_grouped.drop('Borough', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([2, 2, 2, 4, 0, 1, 2, 4, 2, 3], dtype=int32)

## Creating the joint dataframe with clusters and coordinates for the boroughs

In [49]:
boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"[Malvern, Rouge]",43.806686,-79.194353,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
1,M1C,Scarborough,"[Rouge Hill, Port Union, Highland Creek]",43.784535,-79.160497,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
3,M1G,Scarborough,[Woburn],43.770992,-79.216917,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park


## Creating the map of clusters

In [50]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters

### Cluster 1

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
88,Etobicoke,0,Pizza Place,Sandwich Place,Fast Food Restaurant,Gym,Pharmacy,Grocery Store,Coffee Shop,Liquor Store,Park,Beer Store
89,Etobicoke,0,Pizza Place,Sandwich Place,Fast Food Restaurant,Gym,Pharmacy,Grocery Store,Coffee Shop,Liquor Store,Park,Beer Store
90,Etobicoke,0,Pizza Place,Sandwich Place,Fast Food Restaurant,Gym,Pharmacy,Grocery Store,Coffee Shop,Liquor Store,Park,Beer Store
91,Etobicoke,0,Pizza Place,Sandwich Place,Fast Food Restaurant,Gym,Pharmacy,Grocery Store,Coffee Shop,Liquor Store,Park,Beer Store
92,Etobicoke,0,Pizza Place,Sandwich Place,Fast Food Restaurant,Gym,Pharmacy,Grocery Store,Coffee Shop,Liquor Store,Park,Beer Store
93,Etobicoke,0,Pizza Place,Sandwich Place,Fast Food Restaurant,Gym,Pharmacy,Grocery Store,Coffee Shop,Liquor Store,Park,Beer Store
94,Etobicoke,0,Pizza Place,Sandwich Place,Fast Food Restaurant,Gym,Pharmacy,Grocery Store,Coffee Shop,Liquor Store,Park,Beer Store
95,Etobicoke,0,Pizza Place,Sandwich Place,Fast Food Restaurant,Gym,Pharmacy,Grocery Store,Coffee Shop,Liquor Store,Park,Beer Store
99,Etobicoke,0,Pizza Place,Sandwich Place,Fast Food Restaurant,Gym,Pharmacy,Grocery Store,Coffee Shop,Liquor Store,Park,Beer Store
100,Etobicoke,0,Pizza Place,Sandwich Place,Fast Food Restaurant,Gym,Pharmacy,Grocery Store,Coffee Shop,Liquor Store,Park,Beer Store


### Cluster 2

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,Mississauga,1,Hotel,Coffee Shop,Gym,American Restaurant,Middle Eastern Restaurant,Intersection,Mediterranean Restaurant,Sandwich Place,Fried Chicken Joint,Burrito Place


### Cluster 3

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,2,Coffee Shop,Clothing Store,Restaurant,Bank,Sandwich Place,Japanese Restaurant,Park,Grocery Store,Pizza Place,Sushi Restaurant
18,North York,2,Coffee Shop,Clothing Store,Restaurant,Bank,Sandwich Place,Japanese Restaurant,Park,Grocery Store,Pizza Place,Sushi Restaurant
19,North York,2,Coffee Shop,Clothing Store,Restaurant,Bank,Sandwich Place,Japanese Restaurant,Park,Grocery Store,Pizza Place,Sushi Restaurant
20,North York,2,Coffee Shop,Clothing Store,Restaurant,Bank,Sandwich Place,Japanese Restaurant,Park,Grocery Store,Pizza Place,Sushi Restaurant
21,North York,2,Coffee Shop,Clothing Store,Restaurant,Bank,Sandwich Place,Japanese Restaurant,Park,Grocery Store,Pizza Place,Sushi Restaurant
22,North York,2,Coffee Shop,Clothing Store,Restaurant,Bank,Sandwich Place,Japanese Restaurant,Park,Grocery Store,Pizza Place,Sushi Restaurant
23,North York,2,Coffee Shop,Clothing Store,Restaurant,Bank,Sandwich Place,Japanese Restaurant,Park,Grocery Store,Pizza Place,Sushi Restaurant
24,North York,2,Coffee Shop,Clothing Store,Restaurant,Bank,Sandwich Place,Japanese Restaurant,Park,Grocery Store,Pizza Place,Sushi Restaurant
25,North York,2,Coffee Shop,Clothing Store,Restaurant,Bank,Sandwich Place,Japanese Restaurant,Park,Grocery Store,Pizza Place,Sushi Restaurant
26,North York,2,Coffee Shop,Clothing Store,Restaurant,Bank,Sandwich Place,Japanese Restaurant,Park,Grocery Store,Pizza Place,Sushi Restaurant


### Cluster 4

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,York,3,Fast Food Restaurant,Park,Fried Chicken Joint,Pizza Place,Field,Sandwich Place,Bus Line,Restaurant,Caribbean Restaurant,Brewery
74,York,3,Fast Food Restaurant,Park,Fried Chicken Joint,Pizza Place,Field,Sandwich Place,Bus Line,Restaurant,Caribbean Restaurant,Brewery
80,York,3,Fast Food Restaurant,Park,Fried Chicken Joint,Pizza Place,Field,Sandwich Place,Bus Line,Restaurant,Caribbean Restaurant,Brewery
81,York,3,Fast Food Restaurant,Park,Fried Chicken Joint,Pizza Place,Field,Sandwich Place,Bus Line,Restaurant,Caribbean Restaurant,Brewery
98,York,3,Fast Food Restaurant,Park,Fried Chicken Joint,Pizza Place,Field,Sandwich Place,Bus Line,Restaurant,Caribbean Restaurant,Brewery


### Cluster 5

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
1,Scarborough,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
2,Scarborough,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
3,Scarborough,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
4,Scarborough,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
5,Scarborough,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
6,Scarborough,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
7,Scarborough,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
8,Scarborough,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
9,Scarborough,4,Coffee Shop,Indian Restaurant,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Pharmacy,Park
